In [1]:
from cvxpy import *
import numpy as np
import scipy as sp
from scipy import sparse

In [11]:
from time import perf_counter 

In [24]:
%matplotlib inline

In [57]:
# Discrete time model of a quadcopter
# The following two matrices are obtained after performing system 
# identification and state space linearization on the Crazyflie 2.0 quadcopter
Ad = sparse.csc_matrix([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  , -0.  ,  0.  , -0.  ,  0.  ,  1.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  1.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  1.  , -0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  9.81,  0.  ,  0.  ,  0.  , -0.  ,
         0.  , -0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -9.81, -0.  ,  0.  ,  0.  ,
         0.  ,  0.  , -0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.  ,  0.  ,
        -0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  , -0.  , -0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        -0.  , -0.  ,  0.  ]])
Bd = sparse.csc_matrix([[    0.   ,     0.   ,     0.   ,     0.   ],
       [    0.   ,     0.   ,     0.   ,     0.   ],
       [    0.   ,     0.   ,     0.   ,     0.   ],
       [    0.   ,     0.   ,     0.   ,     0.   ],
       [    0.   ,     0.   ,     0.   ,     0.   ],
       [    0.   ,     0.   ,     0.   ,     0.   ],
       [    0.   ,     0.   ,     0.   ,     0.   ],
       [    0.   ,     0.   ,     0.   ,     0.   ],
       [    0.   ,     0.   ,     0.   ,    31.746],
       [56908.235,     0.   ,     0.   ,     0.   ],
       [    0.   , 53850.885,     0.   ,     0.   ],
       [    0.   ,     0.   , 36819.13 ,     0.   ]])
[nx, nu] = Bd.shape

# Constraints

umin = np.array([-0.01, -0.01, -0.01, -0.01]) 
umax = np.array([2., 2., 2., 2.])
xmin = np.array([-np.inf,-np.inf,-np.inf,-np.pi/6,-np.pi/6,-np.pi/6,
                 -2.,-2.,-2.,-1.,-1.,-1.])
xmax = np.array([np.inf,np.inf,np.inf,np.pi/6,np.pi/6,np.pi/6,
                 20.,20.,20.,1.,1.,1.])
#x = [px, py, pz, psi, theta, psi, vx, vy, vz, wx, wy, wz]
#u = [tau_x, tau_y, tau_z, f_z]
# Objective function
Q = sparse.diags([10., 10., 10., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
QN = Q
R = 0.1*sparse.eye(4)

# Initial and reference states
x0 = np.zeros(12)
x0[2] = 0.5

xr = np.array([15., 15., 1.2, 0.,0.,0.,0.,0.,0.,0.,0.,0.])

# Prediction horizon
N = 100

# Define problem
u = Variable((nu, N))
x = Variable((nx, N+1))
x_init = Parameter(nx)

objective = 0
constraints = [x[:,0] == x_init]

for k in range(N):
    objective += quad_form(x[:,k] - xr, Q) + quad_form(u[:,k], R)
    constraints += [x[:,k+1] == Ad@x[:,k] + Bd@u[:,k]]
    constraints += [xmin <= x[:,k], x[:,k] <= xmax]
    constraints += [umin <= u[:,k], u[:,k] <= umax]
   

objective += quad_form(x[:,N] - xr, QN)
prob = Problem(Minimize(objective), constraints)

# Simulate in closed loop
t0 = perf_counter()
nsim = 50

for i in range(nsim):
    x_init.value = x0
    
    prob.solve(solver=OSQP, warm_start=True)
    x0 = Ad.dot(x0) + Bd.dot(u[:,0].value)
    print(x0)
print(perf_counter()-t0)

[ 0.          0.          0.          0.          0.          0.
  0.          0.          1.09089925 -0.52359878  0.52359878  0.        ]
[ 0.          0.          1.09089925  0.          0.52359878 -0.52359878
  0.          0.          1.09089925 -0.52359878  0.52359878  0.        ]
[ 0.          0.          1.09089925  0.          0.52359878 -0.52359878
  5.13650399  5.13650399  1.09089925 -0.52359878  0.52359878  0.        ]
[ 5.13650399  5.13650399  1.09089925  0.          0.52359878 -0.52359878
  5.13650399  5.13650399  1.09089925 -0.52359878  0.52359878  0.        ]
[ 5.13650399  5.13650399  1.09089925  0.          0.52359878 -0.52359878
  5.13650399  5.13650399  1.09089925 -0.52359878  0.52359878  0.        ]
[ 5.13650399  5.13650399  1.09089925  0.          0.52359878 -0.52359878
  5.13650399  5.13650399  1.09089925 -0.52359878  0.52359878  0.        ]
[ 5.13650399  5.13650399  1.09089925  0.          0.52359878 -0.52359878
  5.13650399  5.13650399  1.09089925 -0.52359878  0.5